In [1]:
import geemap
import ee
import pandas as pd
from pandas.io.json import json_normalize
import json

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AbUR2VPVMOPdLPkWkzB6zQr_aQFTYNjm5eihI1BQWDr1LE2ET7FVTs7hOlo



Successfully saved authorization token.


In [2]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon], row[lat]]
        for prop in properties:
            if prop in df.columns:
                feature['properties'][prop] = row[prop]
            else:
                feature['properties'][prop] = None
        geojson['features'].append(feature)
    return geojson


In [3]:
data = pd.read_csv('K:/Google Drive/DATA_SCIENCE/Spectroscopy/OSSL/Kellogg Soil Survey Laboratory/dataset.csv')

data.head()

,Unnamed: 0,id.location_olc_c,id.layer_uuid_c,observation.ogc.schema.title_ogc_txt,observation.ogc.schema_idn_url,observation.date.begin_iso.8601_yyyy.mm.dd,observation.date.end_iso.8601_yyyy.mm.dd,location.address_utf8_txt,location.country_iso.3166_c,location.method_any_c,...,scan_mir.3982_abs,scan_mir.3984_abs,scan_mir.3986_abs,scan_mir.3988_abs,scan_mir.3990_abs,scan_mir.3992_abs,scan_mir.3994_abs,scan_mir.3996_abs,scan_mir.3998_abs,scan_mir.4000_abs
0,0,854MFJ9X+V8,7461ec5822f31c1071c09bb5ad293bb4,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,13477,13477,White Sands,USA,GPS,...,257.0,257.0,257.0,257.0,257.0,257.0,257.0,257.0,257.0,257.0
1,1,854MFJ9X+V8,696da8718f4b20ac408558e7dfd9f7ee,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,13477,13477,White Sands,USA,GPS,...,333.0,334.0,334.0,334.0,335.0,335.0,336.0,336.0,336.0,337.0
2,2,854MFJ9X+V8,d28bab21a85b3f51bb54f9f78c9b01ed,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,13477,13477,White Sands,USA,GPS,...,241.0,241.0,241.0,241.0,240.0,240.0,240.0,240.0,240.0,240.0
3,3,854MFJ9X+V8,61b28560becdc2c79efd872fdbf12d03,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,13477,13477,White Sands,USA,GPS,...,165.0,165.0,164.0,164.0,163.0,162.0,162.0,161.0,160.0,160.0
4,4,854MFJ9X+V8,a0946d7385a92c63af1fba1f83cb2238,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,13477,13477,White Sands,USA,GPS,...,149.0,149.0,148.0,148.0,147.0,147.0,146.0,145.0,145.0,144.0


In [13]:
df = pd.DataFrame()

df['id.layer_uuid_c'] = data['id.layer_uuid_c']
df['longitude'] = data['longitude_wgs84_dd']
df['latitude'] = data['latitude_wgs84_dd']

df.head()

,id.layer_uuid_c,longitude,latitude
0,7461ec5822f31c1071c09bb5ad293bb4,-106.351722,32.469639
1,696da8718f4b20ac408558e7dfd9f7ee,-106.351722,32.469639
2,d28bab21a85b3f51bb54f9f78c9b01ed,-106.351722,32.469639
3,61b28560becdc2c79efd872fdbf12d03,-106.351722,32.469639
4,a0946d7385a92c63af1fba1f83cb2238,-106.351722,32.469639


In [14]:
print(df.shape)

df = df.query('longitude < -60')

print(df.shape)

(1337, 3)
(1276, 3)


In [15]:
latitude_center = df['latitude'].mean()
longitude_center = df['longitude'].mean()
zoom_level = 4

In [16]:
geojson = df_to_geojson(df, properties=['Property1', 'Property2'])

In [17]:
fc = ee.FeatureCollection(geojson['features'])


In [18]:
Map = geemap.Map(center=[latitude_center, longitude_center], zoom=zoom_level)

Map.addLayer(fc, {}, 'Points')

Map

Map(center=[38.18537747344131, -99.29330688610239], controls=(WidgetControl(options=['position', 'transparent_…